In [ ]:
!pip install --upgrade --force-reinstall pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2025.2
    Uninstalling tzdata-2025.2:
      Successfully uninstalled tzdata-2025.2
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six-1

In [8]:
from google.colab import files
import zipfile
import os
import pandas as pd

# 1. zip 파일 업로드 받기
uploaded = files.upload()  # dataset.zip을 올려주세요!

# 2. 압축 해제
with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")
print("✅ 압축 해제 완료!")

# 3. CSV 생성
DATASET_DIR = "dataset/dataset"
image_paths, object_types, material_types = [], [], []

for obj in os.listdir(DATASET_DIR):
    obj_path = os.path.join(DATASET_DIR, obj)
    if obj in ['cup', 'straw']:
        for material in os.listdir(obj_path):
            mat_path = os.path.join(obj_path, material)
            for fname in os.listdir(mat_path):
                if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                    image_paths.append(os.path.join(obj_path, material, fname))
                    object_types.append(obj)
                    material_types.append(material)
    elif obj == 'cupholder':
        for fname in os.listdir(obj_path):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_paths.append(os.path.join(obj_path, fname))
                object_types.append(obj)
                material_types.append("paper")
    elif obj == 'lid':
        for fname in os.listdir(obj_path):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_paths.append(os.path.join(obj_path, fname))
                object_types.append(obj)
                material_types.append("plastic")

df = pd.DataFrame({
    "image_path": image_paths,
    "object_type": object_types,
    "material_type": material_types
})
df.to_csv("labels.csv", index=False)
print("✅ labels.csv 생성 완료!")

# 4. 다운로드
files.download("labels.csv")


Saving dataset.zip to dataset (1).zip
✅ 압축 해제 완료!
✅ labels.csv 생성 완료!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
import pandas as pd

df = pd.read_csv("/content/labels.csv")  # 경로 주의

object_type_map = {'cup': 0, 'cupholder': 1, 'lid': 2, 'straw': 3}
material_type_map = {'paper': 0, 'plastic': 1}

df['object_label'] = df['object_type'].map(object_type_map)
df['material_label'] = df['material_type'].map(material_type_map)

print(df.head())


                                    image_path object_type material_type  \
0  dataset/dataset/cupholder/cupholder_051.jpg   cupholder         paper   
1  dataset/dataset/cupholder/cupholder_045.jpg   cupholder         paper   
2  dataset/dataset/cupholder/cupholder_053.jpg   cupholder         paper   
3  dataset/dataset/cupholder/cupholder_025.jpg   cupholder         paper   
4  dataset/dataset/cupholder/cupholder_037.jpg   cupholder         paper   

   object_label  material_label  
0             1               0  
1             1               0  
2             1               0  
3             1               0  
4             1               0  


In [10]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df[['object_label', 'material_label']]
)


tf.data.Dataset 만들기

In [11]:
import tensorflow as tf

IMG_SIZE = (180, 180)
BATCH_SIZE = 32

def preprocess(image_path, object_label, material_label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0
    return image, {'object_output': object_label, 'material_output': material_label}

def create_dataset(dataframe, shuffle=True):
    paths = dataframe['image_path'].values
    obj_labels = dataframe['object_label'].values
    mat_labels = dataframe['material_label'].values

    dataset = tf.data.Dataset.from_tensor_slices((paths, obj_labels, mat_labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_ds = create_dataset(train_df)
val_ds = create_dataset(val_df, shuffle=False)


데이터 증강 레이어 (훈련용에만 적용)

In [12]:
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

def augment(image, labels):
    return data_augmentation(image, training=True), labels

train_ds = train_ds.map(augment, num_parallel_calls=tf.data.AUTOTUNE)


모델 구성 (MobileNetV2 + Multi-output)

In [13]:
from tensorflow.keras import models

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(180, 180, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

inputs = tf.keras.Input(shape=(180, 180, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)

object_output = layers.Dense(4, activation='softmax', name='object_output')(x)
material_output = layers.Dense(2, activation='softmax', name='material_output')(x)

model = models.Model(inputs=inputs, outputs={'object_output': object_output, 'material_output': material_output})


/tmp/ipython-input-13-95944343.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


컴파일 & 학습

In [14]:
model.compile(
    optimizer='adam',
    loss={
        'object_output': 'sparse_categorical_crossentropy',
        'material_output': 'sparse_categorical_crossentropy'
    },
    metrics={
        'object_output': 'accuracy',
        'material_output': 'accuracy'
    }
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - loss: 1.6479 - material_output_accuracy: 0.6746 - material_output_loss: 0.7314 - object_output_accuracy: 0.6493 - object_output_loss: 0.9075 - val_loss: 0.2872 - val_material_output_accuracy: 0.8889 - val_material_output_loss: 0.2129 - val_object_output_accuracy: 1.0000 - val_object_output_loss: 0.0726
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - loss: 0.1924 - material_output_accuracy: 0.9749 - material_output_loss: 0.1196 - object_output_accuracy: 0.9883 - object_output_loss: 0.0718 - val_loss: 0.1067 - val_material_output_accuracy: 0.9444 - val_material_output_loss: 0.0991 - val_object_output_accuracy: 1.0000 - val_object_output_loss: 0.0107
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - loss: 0.0466 - material_output_accuracy: 1.0000 - material_output_loss: 0.0337 - object_output_accuracy: 1.0000 - object_output_loss: 0.0126 - val_loss: 0.0477 - val_material_output_accuracy: 0.9815 - val_material_output_loss: 0.0457 - val_ob

In [15]:
model.save("/content/multilabel_model.keras")  # ✅ 권장 포맷
print("✅ 모델 저장 완료!")


✅ 모델 저장 완료!


In [16]:
OBJECT_CLASS_NAMES = ['cup', 'cupholder', 'lid', 'straw']
MATERIAL_CLASS_NAMES = ['paper', 'plastic']


In [18]:
from tensorflow.keras.preprocessing import image
import numpy as np
from pathlib import Path

def predict_image(img_path):
    if not Path(img_path).exists():
        print(f"❌ 파일을 찾을 수 없습니다: {img_path}")
        return

    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array / 255.0, axis=0)  # 정규화

    preds = model.predict(img_array)

    object_probs = preds['object_output'][0]
    material_probs = preds['material_output'][0]

    object_label = np.argmax(object_probs)
    material_label = np.argmax(material_probs)

    print(f"🧠 물체 예측: {OBJECT_CLASS_NAMES[object_label]} ({object_probs[object_label]*100:.2f}%)")
    print(f"🧪 재질 예측: {MATERIAL_CLASS_NAMES[material_label]} ({material_probs[material_label]*100:.2f}%)")


In [20]:
predict_image("/content/dataset/dataset/cup/paper/cup_paper_001.jpg")
predict_image("/content/dataset/dataset/lid/lid_001.jpg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
🧠 물체 예측: cup (99.92%)
🧪 재질 예측: paper (99.84%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
🧠 물체 예측: lid (100.00%)
🧪 재질 예측: plastic (100.00%)


In [21]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_and_show(img_path):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array / 255.0, axis=0)

    preds = model.predict(img_array)

    object_probs = preds['object_output'][0]
    material_probs = preds['material_output'][0]

    object_label = np.argmax(object_probs)
    material_label = np.argmax(material_probs)

    plt.imshow(img.astype("uint8"))
    plt.axis("off")
    plt.title(f"🧠 {OBJECT_CLASS_NAMES[object_label]} ({object_probs[object_label]*100:.1f}%)\n🧪 {MATERIAL_CLASS_NAMES[material_label]} ({material_probs[material_label]*100:.1f}%)")
    plt.show()


In [23]:
predict_image("/content/dataset/dataset/cup/paper/cup_paper_013.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
🧠 물체 예측: cup (99.97%)
🧪 재질 예측: paper (99.47%)
